# AOG Image Style Transfer Example

This notebook demonstrates how to use AOG Image-to-Image API for image style transfer and editing.

⚠️ **Important**: This service typically uses cloud API (such as Alibaba Cloud Tongyi Wanxiang), requires pre-configuration

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Image Transformation](#3-Basic Image Transformation)
4. [Different Style Transformations](#4-Different Style Transformations)
5. [Batch Processing](#5-Batch Processing)

## 1. Environment Setup

In [ ]:
import requests
import json
import base64
from typing import Optional, Dict, List
from pathlib import Path
from IPython.display import Image, display
import os

# AOG service configuration
AOG_BASE_URL = "http://localhost:16688"
AOG_API_SPEC_VERSION = "v0.2"
DEFAULT_MODEL = "wanx2.1-i2i-turbo"

print("✅ Configuration complete")
print(f"📍 AOG service address: {AOG_BASE_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL}")
print("☁️  Cloud service: Alibaba Cloud Tongyi Wanxiang")

## 2. Helper Functions

In [ ]:
def encode_image_to_base64(image_path: str) -> Optional[str]:
    """
    将图像文件编码为 base64 字符串
    
    Args:
        image_path: 图像文件路径
    
    Returns:
        base64 编码字符串
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"❌ 编码图像失败: {e}")
        return None

def call_aog_image_to_image(
    image: str,
    image_type: str = "path",
    prompt: str = "",
    model: str = DEFAULT_MODEL,
    n: int = 1
) -> Optional[Dict]:
    """
    Call AOG Image-to-Image API
    
    Args:
        image: 输入图像(路径/URL/base64)
        image_type: 图像类型 (path/url/base64)
        prompt: 描述期望风格或变化
        model: Model name
        n: 生成图像数量
    
    Returns:
        API response data
    """
    try:
        url = f"{AOG_BASE_URL}/aog/{AOG_API_SPEC_VERSION}/services/image-to-image"
        
        payload = {
            "model": model,
            "image": image,
            "image_type": image_type,
            "n": n
        }
        
        if prompt:
            payload["prompt"] = prompt
        
        print(f"📤 Sending request...")
        print(f"🖼️  图像类型: {image_type}")
        if prompt:
            print(f"📝 Tip词: {prompt}")
        print(f"🔢 生成数量: {n}")
        
        response = requests.post(url, json=payload, timeout=120)
        response.raise_for_status()
        
        result = response.json()
        
        if result.get("business_code") == 200:
            print("✅ 图像Conversion successful")
            urls = result.get("data", {}).get("url", [])
            print(f"🎨 生成了 {len(urls)} 张图像")
        else:
            print(f"⚠️  业务错误: {result.get('message')}")
        
        return result
        
    except requests.exceptions.ConnectionError:
        print("❌ 无法连接到 AOG 服务")
        return None
    except requests.exceptions.Timeout:
        print("❌ Request timeout(图像转换可能需要较长时间)")
        return None
    except Exception as e:
        print(f"❌ 错误: {e}")
        return None

def display_images(image_urls: List[str]):
    """
    在 notebook 中显示图像
    
    Args:
        image_urls: 图像 URL 或路径列表
    """
    if not image_urls:
        print("⚠️  没有图像可显示")
        return
    
    for i, url in enumerate(image_urls, 1):
        print(f"\n🖼️  图像 {i}/{len(image_urls)}:")
        try:
            if url.startswith('http'):
                # 云端 URL
                print(f"🔗 URL: {url}")
                display(Image(url=url, width=512))
            else:
                # 本地路径
                print(f"📁 路径: {url}")
                if Path(url).exists():
                    display(Image(filename=url, width=512))
                else:
                    print(f"⚠️  文件不存在: {url}")
        except Exception as e:
            print(f"❌ 显示失败: {e}")

def download_image(url: str, save_path: str):
    """
    下载图像到本地
    
    Args:
        url: 图像 URL
        save_path: 保存路径
    """
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ 图像已保存到: {save_path}")
    except Exception as e:
        print(f"❌ 下载失败: {e}")

print("✅ Helper Functions定义完成")

## 3. Basic Image Transformation

使用本地图像for风格转换

In [ ]:
# 准备输入图像
# 请替换为您自己图像路径
input_image_path = "path/to/your/image.jpg"

# 检查文件是否存在
if not Path(input_image_path).exists():
    print(f"⚠️  请先准备一张图像,并将路径设置为: {input_image_path}")
    print("💡 Tip: 您可以使用任何 JPG、PNG 等格式图像")
else:
    print("✅ 找到输入图像")
    print(f"📁 路径: {input_image_path}")
    
    # 显示原始图像
    print("\n🖼️  原始图像:")
    display(Image(filename=input_image_path, width=512))

In [ ]:
# 基本转换示例 - 油画风格
prompt = "改为油画风格"

response = call_aog_image_to_image(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("business_code") == 200:
    image_urls = response.get("data", {}).get("url", [])
    display_images(image_urls)

## 4. Different Style Transformations

尝试多种艺术风格

In [ ]:
# 水彩画风格
prompt = "转换为水彩画风格"

response = call_aog_image_to_image(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("business_code") == 200:
    image_urls = response.get("data", {}).get("url", [])
    display_images(image_urls)

In [ ]:
# 卡通风格
prompt = "改为卡通风格"

response = call_aog_image_to_image(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("business_code") == 200:
    image_urls = response.get("data", {}).get("url", [])
    display_images(image_urls)

In [ ]:
# 素描风格
prompt = "转换为素描风格"

response = call_aog_image_to_image(
    image=input_image_path,
    image_type="path",
    prompt=prompt
)

if response and response.get("business_code") == 200:
    image_urls = response.get("data", {}).get("url", [])
    display_images(image_urls)

## 5. Batch Processing

一次生成多张不同风格图像

In [ ]:
# 定义多种风格
styles = [
    "改为油画风格",
    "转换为水彩画风格",
    "改为卡通风格",
    "转换为素描风格"
]

all_results = []

for i, style in enumerate(styles, 1):
    print(f"\n🎨 处理风格 {i}/{len(styles)}: {style}")
    print("=" * 50)
    
    response = call_aog_image_to_image(
        image=input_image_path,
        image_type="path",
        prompt=style
    )
    
    if response and response.get("business_code") == 200:
        image_urls = response.get("data", {}).get("url", [])
        all_results.append({
            "style": style,
            "urls": image_urls
        })
        display_images(image_urls)

print(f"\n✅ 成功生成 {len(all_results)}/{len(styles)} 种风格")

## 6. 使用 URL 输入

直接使用网络图像 URL

In [ ]:
# 使用网络图像 URL
image_url = "https://example.com/your-image.jpg"
prompt = "改为油画风格"

response = call_aog_image_to_image(
    image=image_url,
    image_type="url",
    prompt=prompt
)

if response and response.get("business_code") == 200:
    image_urls = response.get("data", {}).get("url", [])
    display_images(image_urls)

## 7. 下载生成图像

将云端图像下载到本地

In [ ]:
# 下载最近生成图像
if all_results:
    # 创建输出Table of Contents
    output_dir = Path("output_images")
    output_dir.mkdir(exist_ok=True)
    
    for result in all_results:
        style = result["style"]
        urls = result["urls"]
        
        for i, url in enumerate(urls):
            if url.startswith('http'):
                # 生成文件名
                filename = f"{style.replace(' ', '_')}_{i+1}.png"
                save_path = output_dir / filename
                
                print(f"\n📥 下载: {style}")
                download_image(url, str(save_path))
    
    print(f"\n✅ 所有图像已保存到: {output_dir}")
else:
    print("⚠️  没有可下载图像")

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ 调用 Image-to-Image API
2. ✅ 使用不同图像输入方式(路径/URL/Base64)
3. ✅ 应用多种艺术风格
4. ✅ Batch Processing图像
5. ✅ 下载和保存生成图像

### Tip词建议

**风格转换**:
- 改为油画风格
- 转换为水彩画风格
- 改为卡通风格
- 转换为素描风格

**图像编辑**:
- 增加更多细节
- 改变背景为夜晚
- 添加更多色彩
- 使图像更加明亮

### Next Steps

- 尝试更多创意Tip词
- 探索 [Image-to-Video](../image-to-video/) 服务
- Check out [Text-to-Image](../text-to-image/) 服务
- Read [AOG API 文档](../../docs/)